<a href="https://colab.research.google.com/github/Heli9x/VG-AI-GEN-01/blob/main/VoiceGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Python packages
!pip install flask pyngrok nest-asyncio -qq

# 2. Install the ngrok executable
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!chmod +x ngrok
!mv ngrok /usr/local/bin/
# 3. Authenticate ngrok (Use the old 'authtoken' command for this version)
# 🚨 REPLACE YOUR_NGROK_AUTH_TOKEN with your actual token
!ngrok authtoken 34jmqhKXbQK4AKRcqpQ8YuN8TJY_5WputRo1qCN3oNSks3bGE

--2025-10-30 12:16:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  12.2MB/s    in 1.1s    

2025-10-30 12:16:24 (12.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, request, send_file
import torch
import numpy as np
import scipy.io.wavfile
from transformers import AutoProcessor, BarkModel
import os
import nest_asyncio
from pyngrok import ngrok # Import ngrok for the tunnel

class VoiceGen:
    def __init__(self):
        """Initialize the Bark model and processor."""
        print("Loading Bark model...")
        self.processor = AutoProcessor.from_pretrained("suno/bark")
        self.model = BarkModel.from_pretrained(
            "suno/bark",
            torch_dtype=torch.float16,
            device_map="auto",
            offload_folder="offload"
        )

        # Set pad_token_id if not already set
        if self.model.config.pad_token_id is None:
            self.model.config.pad_token_id = self.model.config.eos_token_id

        print(f"Model loaded on device: {self.model.device}")

    def generate_voice(self, text, output_name, voice_preset="v2/en_speaker_6"):
        """Generate voice audio from text, returns output path."""
        try:
            print(f"Processing text: '{text[:50]}...'")

            inputs = self.processor(
                text,
                voice_preset=voice_preset
            )

            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

            print("Generating audio...")
            with torch.no_grad():
                audio_array = self.model.generate(**inputs)

            audio_array = audio_array.cpu().numpy().squeeze()
            audio_array = (audio_array * 32767).astype(np.int16)
            sample_rate = self.model.generation_config.sample_rate

            output_path = f"{output_name}.wav"
            scipy.io.wavfile.write(output_path, rate=sample_rate, data=audio_array)

            print(f"✅ Done! Audio saved as {output_path}")
            return output_path

        except Exception as e:
            print(f"❌ Error generating voice: {str(e)}")
            # Raise the exception for the Flask app to catch and return a 500 error
            raise


# --- Flask App Setup ---
app = Flask(__name__)
voice_gen = VoiceGen() # Initialize VoiceGen (loads the model)

@app.route("/generate_tts", methods=["POST"])
def generate_tts():
    # Retrieve data from JSON body
    data = request.get_json()
    if not data or "text" not in data:
        return {"error": "Invalid request. 'text' field is required."}, 400

    text = data["text"]
    output_name = data.get("output_name", "generated_audio")
    voice_preset = data.get("voice_preset", "v2/en_speaker_6")

    try:
        # Generate the WAV file
        output_path = voice_gen.generate_voice(text, output_name, voice_preset)

        # Send the file back to the user and then delete it
        return send_file(
            output_path,
            as_attachment=True,
            mimetype="audio/wav",
            download_name=f"{output_name}.wav"
        )
    finally:
        # Clean up the file after sending (best practice)
        if 'output_path' in locals() and os.path.exists(output_path):
             os.remove(output_path)

Loading Bark model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Model loaded on device: cuda:0


In [ ]:
# Apply nest_asyncio to allow the web server to run inside Colab's event loop
nest_asyncio.apply()

# --- Start ngrok tunnel (must happen before the blocking app.run) ---
NGROK_PORT = 5000
try:
    # 1. Terminate any previous ngrok sessions in case the cell was rerun
    ngrok.kill()

    # 2. 🚨 CRITICAL FIX: Set the auth token directly via the Python API
    #    REPLACE THIS WITH YOUR ACTUAL TOKEN
    NGROK_AUTH_TOKEN = "34jmqhKXbQK4AKRcqpQ8YuN8TJY_5WputRo1qCN3oNSks3bGE"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # 3. Open a new tunnel
    tunnel = ngrok.connect(NGROK_PORT)
    public_url = tunnel.public_url

    print("=" * 50)
    print(f"🎉 Public API Endpoint (POST): {public_url}/generate_tts")
    print(f"   Curl Example: curl -X POST {public_url}/generate_tts -H 'Content-Type: application/json' -d '{{\"text\": \"Hello world\"}}' --output out.wav")
    print("=" * 50)

    # 4. Run the Flask app (This is a blocking call and must be the last line)
    app.run(host="0.0.0.0", port=NGROK_PORT, debug=False, use_reloader=False)

except Exception as e:
    print(f"Failed to start server or ngrok: {e}")

🎉 Public API Endpoint (POST): https://hypodermal-feignedly-amia.ngrok-free.dev/generate_tts
   Curl Example: curl -X POST https://hypodermal-feignedly-amia.ngrok-free.dev/generate_tts -H 'Content-Type: application/json' -d '{"text": "Hello world"}' --output out.wav
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Processing text: 'Kenny is a bad girl...'


speaker_embeddings/v2/en_speaker_7_seman(…):   0%|          | 0.00/2.32k [00:00<?, ?B/s]

speaker_embeddings/v2/en_speaker_7_coars(…):   0%|          | 0.00/6.70k [00:00<?, ?B/s]

...

✅ Done! Audio saved as generated_audio.wav
Processing text: 'How fast can i go? [think] Maybe 50 kilometres...?...'


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Generating audio...


INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 12:25:21] "POST /generate_tts HTTP/1.1" 200 -


✅ Done! Audio saved as generated_audio.wav
